<a href="https://colab.research.google.com/github/zia207/r-colab/blob/main/NoteBook/Advance_Regression/01_Generalized_Linear_Models/02-01-09-00-glm-gamlss-introduction-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://drive.google.com/uc?export=view&id=1IFEWet-Aw4DhkkVe1xv_2YYqlvRe9m5_)

# 9. Generalized Additive Model for Location, Scale, and Shape (GAMLSS)

**GAMLSS (Generalized Additive Models for Location Scale and Shape)** are distributional regression models that allow explanatory variables (X) to affect all aspects of the response variable's distribution (y), not just the expected value. This is useful for analyzing shifts in not only the mean but also variance, skewness, kurtosis, and quantiles.

GAMLSS can be implemented in Python via packages such as `ondil`, which provides a native implementation for online distributional learning with GAMLSS support. Other related packages include `pygam` for standard GAMs (which can be extended conceptually) and `statsmodels` for foundational regression tools. The `ondil` package emphasizes efficiency with just-in-time compilation via `numba` and compatibility with `scikit-learn`. It supports a variety of distributions and regularization methods. GAMLSS offers over 100 continuous, discrete, and mixed distributions for response modeling in R, but in Python via `ondil`, key distributions like Gaussian, Student's t, Johnson's SU, and others are available, with extensibility for more. The book *Distributions for Modelling Location, Scale, and Shape: Using GAMLSS in R* remains a comprehensive guide, adaptable to Python contexts.

Within GAMLSS, machine learning techniques can model distribution parameters. Options for additive smoothing include P-splines and cubic smoothing splines (via extensions or custom implementations), while methods for dimensionality reduction encompass ridge regression and LASSO. Interaction modeling can be achieved with regression trees and neural networks (integrable via `scikit-learn`), and random effects can also be incorporated.

GAMLSS is particularly beneficial for practitioners handling skewed and kurtotic data, common in fields like medicine and finance. Notable users include the World Health Organization (WHO), the Global Lung Function Initiative, the International Monetary Fund (IMF), and the European Parliament, among others.

## Overview

In GAMLSS, the response variable ( Y ) is assumed to follow a parametric distribution with probability density function (pdf) or probability mass function (pmf):
$$
Y \sim D(\mu, \sigma, \nu, \tau, \dots)
$$
where the distribution $D$ has up to **four parameters**:
- $\mu$: **Location** (e.g., mean)
- $\sigma$: **Scale** (e.g., standard deviation)
- $\nu$: **Shape** (e.g., skewness)
- $\tau$: **Shape** (e.g., kurtosis)

Each parameter is modeled via a **link function** as an additive function of explanatory variables:
$$
\begin{align*}
g_1(\mu) &= \eta_1 = \beta_{10} + f_{11}(x_1) + f_{12}(x_2) + \dots + f_{1p}(x_p) \\
g_2(\sigma) &= \eta_2 = \beta_{20} + f_{21}(x_1) + f_{22}(x_2) + \dots + f_{2p}(x_p) \\
g_3(\nu) &= \eta_3 = \beta_{30} + f_{31}(x_1) + f_{32}(x_2) + \dots + f_{3p}(x_p) \\
g_4(\tau) &= \eta_4 = \beta_{40} + f_{41}(x_1) + f_{42}(x_2) + \dots + f_{4p}(x_p)
\end{align*}
$$
where:
- $g_k(\cdot)$ are **link functions** (e.g., identity, log, logit)
- $\eta_k$ are **linear predictors** for the $k$-th parameter
- $f_{kj}(\cdot)$ are **smooth functions** (splines, polynomials, etc.) of covariates
- $\beta_{k0}$ are intercepts

This framework allows **nonlinear, nonparametric relationships** between predictors and each distribution parameter.

### Key Features of GAMLSS

1. **Flexible Distribution Assumptions**
    - Can model responses using a wide range of distributions, including highly skewed, heavy-tailed, zero-inflated, or bounded distributions (e.g., Student's t, Johnson's SU, Beta, via `ondil.distributions`).
    - Not limited to exponential family (unlike GLMs/GAMs).

2. **Modeling All Distribution Parameters**
    - Unlike standard models that only model the mean, GAMLSS can model **variance, skewness, and kurtosis** as functions of covariates.
    - Enables **heteroscedasticity modeling** and **distributional regression**.

3. **Additive Predictors**
    - Each parameter is modeled as an **additive function** of covariates, similar to GAMs.
    - Can include smooth terms, random effects, spatial effects, and interactions (via `scikit-learn` integration).

4. **Robustness to Non-Normality**
    - Handles data that violate normality, constant variance, or other classical assumptions.

5. **Estimation via Maximum Likelihood**
    - Parameters are estimated using **(penalized) maximum likelihood** with online coordinate descent in `ondil`.
    - Smoothing parameters are selected using criteria like AIC, BIC, or GCV (via `ic` parameter).

6. **Software Support**
    - Implemented in Python via the `ondil` package (Hirsch et al., ongoing development).

### Applications of GAMLSS

GAMLSS is widely used in fields where data exhibit complex distributional behavior:

1. **Health and Medicine**
    - Modeling growth charts (e.g., WHO child growth standards), where percentiles (via ( \mu ) and ( \sigma )) vary with age.
    - Blood pressure, BMI, or biomarker modeling with age, sex, and lifestyle factors.

2. **Insurance and Risk Modeling**
    - Modeling claim sizes with heavy-tailed distributions (e.g., Pareto, GB2 approximations via custom distributions).
    - Actuarial risk assessment with variable scale and shape.

3. **Environmental Science**
    - Rainfall, temperature, or pollution levels with skewness and heteroscedasticity.

4. **Economics and Finance**
    - Income distribution modeling (skewed, heavy-tailed).
    - Volatility modeling (scale as a function of predictors).

5. **Ecology**
    - Species abundance with zero-inflation and overdispersion.

### How GAMLSS Differs from Standard GAM

| Feature                  | **Standard GAM**                                                                 | **GAMLSS**                                                                 |
|--------------------------|----------------------------------------------------------------------------------|----------------------------------------------------------------------------|
| **Distribution**         | Typically exponential family (Normal, Poisson, Binomial)                         | Any parametric distribution (including non-exponential family)              |
| **Parameters Modeled**   | Only the **mean** (location)                                                     | **All parameters**: mean, scale, skewness, kurtosis                         |
| **Variance Assumption**  | Often assumes constant variance or variance linked to mean                       | **Variance (and shape) can depend on covariates**                           |
| **Flexibility**          | Flexible mean function                                                           | Flexible **entire distribution**                                            |
| **Modeling Goal**        | Estimate conditional mean                                                        | Estimate full conditional distribution                                      |
| **Example Use Case**     | Predicting average house price (using `pygam`)                                   | Predicting full price distribution (mean, spread, skew) using `ondil`       |

### The {ondil} Package Ecosystem

The GAMLSS framework in Python is primarily supported by `ondil`, with complementary packages for broader functionality:

1. The core `{ondil}` package for fitting online GAMLSS models.
2. Integration with `{scikit-learn}` for preprocessing, regularization, and extensions.
3. `{pygam}` for standard GAMs, useful for comparison or baseline mean modeling.
4. `{statsmodels}` for additional distributional tools and diagnostics.
5. `{numba}` (dependency) for performance optimization.
6. Custom extensions for new distributions or smoothers.

### Main Functions

| Function/Class/Method                  | Description                                                                 |
|----------------------------------------|-----------------------------------------------------------------------------|
| `OnlineDistributionalRegression`       | Main class for fitting GAMLSS models, specifying distribution, method (e.g., Lasso), and equations for parameters. |
| `fit(X, y)`                            | Fits the model on initial data (similar to `gamlss()` but online-capable).  |
| `update(X, y)`                         | Incrementally updates the model with new data.                              |
| `predict_distribution_parameters(X)`   | Predicts parameters (e.g., μ, σ) for new data (equivalent to `predict()`).  |
| `beta`                                 | Accesses fitted coefficients (via attribute; for summary-like output).      |
| `ic` (parameter)                       | Specifies information criterion (AIC/BIC) for selection.                    |
| Residuals (custom)                     | Compute normalized quantile residuals using `scipy` or custom functions.    |
| `summary` (custom/print)               | Print model summary via `print(model.beta)` or custom logging.              |
| Plots (via `matplotlib`)               | Generate diagnostic plots (Q-Q, density) post-prediction.                   |
| `wp()` (custom)                        | Worm plots for residuals using visualization libraries.                     |
| Centiles (custom)                      | Compute and plot centile curves from predicted distributions.               |
| `AIC` / `BIC` (via `ic`)               | Calculates information criteria during fitting.                             |
| Model selection (manual)               | Stepwise selection via iterative fitting/updating.                          |
| `histDist` (custom via `seaborn`)      | Fit and plot parametric distributions to histograms.                        |
| `pdf.plot` (custom)                    | Plot probability density functions using `scipy.stats`.                     |
| `rqres.plot` (custom)                  | Plot randomized quantile residuals.                                         |
| Hyperparameter tuning (via grid)       | Optimize via cross-validation with `scikit-learn`.                          |

### Example: Fitting a Simple GAMLSS Model in Python

To illustrate, here's a basic example using the `ondil` package to fit a GAMLSS model with a Student's t distribution on the diabetes dataset:

In [4]:
!pip install ondil

In [7]:
import numpy as np
from sklearn.datasets import load_diabetes
from ondil.estimators import OnlineDistributionalRegression
from ondil.distributions import StudentT
from scipy.stats import t

# Load data
X, y = load_diabetes(return_X_y=True)

# Define equations for distribution parameters
equation = {
    0: list(range(X.shape[1])),  # All features for mu
    1: list(range(X.shape[1])),  # All features for sigma
    2: list(range(X.shape[1]))   # All features for nu
}

# Initialize and fit
model = OnlineDistributionalRegression(
    distribution=StudentT(),
    method="lasso",
    equation=equation,
    fit_intercept=True,
    ic="bic"
)

# Fit on training data
n_train = int(0.8 * len(y))
model.fit(X=X[:n_train], y=y[:n_train])

# Predict parameters
test_params = model.predict_distribution_parameters(X=X[n_train:])
print("Type of test_params:", type(test_params))
print("Shape of test_params:", test_params.shape)

# Extract loc, scale, df assuming test_params is (n_test, 3)
loc = test_params[:, 0]   # Location (mu)
scale = test_params[:, 1] # Scale (sigma)
df = test_params[:, 2]    # Shape (nu, degrees of freedom)

# Verify shapes
print("Shape of loc:", loc.shape)
print("Shape of scale:", scale.shape)
print("Shape of df:", df.shape)

# Compute residuals
q = (np.arange(len(y[n_train:])) + 0.5) / len(y[n_train:])
residuals = t.ppf(q, df=df, loc=loc, scale=scale)
print("Sample residuals:", residuals[:5])

Type of test_params: <class 'numpy.ndarray'>
Shape of test_params: (89, 3)
Shape of loc: (89,)
Shape of scale: (89,)
Shape of df: (89,)
Sample residuals: [ 46.75285597  88.81236212  28.67867461   1.64181304 112.03535212]


This fits a model where all parameters depend on the full set of predictors, regularized via Lasso. Predictions return arrays of [location, scale, shape] for each observation. Extend with smooth terms by preprocessing X with splines from `pygam` or `scipy`.

## Resources

1. Rigby, R. A., & Stasinopoulos, D. M. (2005). *Generalized additive models for location, scale and shape*. Journal of the Royal Statistical Society: Series C (Applied Statistics), 54(3), 507–554.
2. Stasinopoulos, D. M., & Rigby, R. A. (2007). *Generalized additive models for location scale and shape (GAMLSS) in R*. Journal of Statistical Software, 23(7), 1–46.
3. [ondil Documentation](https://simon-hirsch.github.io/ondil/) - Official Python GAMLSS implementation with examples.
4. [Flexible Regression and Smoothing: Using GAMLSS in R](https://www.taylorfrancis.com/books/mono/10.1201/b21973/flexible-regression-smoothing-mikis-stasinopoulos-robert-rigby-gillian-heller-vlasios-voudouris-fernanda-de-bastiani) (adaptable concepts).
5. [Distributions for Location Scale and Shape: Using GAMLSS in R](https://www.taylorfrancis.com/books/mono/10.1201/9780429298547/distributions-modeling-location-scale-shape-robert-rigby-mikis-stasinopoulos-gillian-heller-fernanda-de-bastiani).
6. [Generalised Additive Models for Location Scale and Shape: A Distributional Regression Approach with Applications](https://www.cambridge.org/core/books/generalized-additive-models-for-location-scale-and-shape/3499AA3827369F287B604639D1127FE3/).